In [ ]:
from Converters import parse_cvi42_xml as wsx

In [ ]:
wsx.parseFile('./pericardial/perifat1_cont.cvi42wsx')

In [ ]:
import json

In [ ]:
d = json.load(open('pericardial/perifat1_cont_contours.json','r'))

In [ ]:
import pickle

In [ ]:
with open('./pericardial/1.3.12.2.1107.5.2.18.41754.2015081410592855489077245.pickle','rb') as f:
    a = pickle.load(f)

In [ ]:
with open('./pericardial/1.3.12.2.1107.5.2.18.41754.201508141059285567277192.pickle','rb') as f:
    b = pickle.load(f)

In [ ]:
with open('./pericardial/perifat1_cont_contours_dct.pickle','rb') as f:
    c = pickle.load(f)

In [ ]:
c.keys()

In [ ]:
import matplotlib.pyplot as plt

import matplotlib.path as mpath

In [ ]:
contour = a['freeDrawRoiContour']

In [ ]:
import numpy as np

In [ ]:
plt.scatter(contour[:,0],contour[:,1])

plt.axis('equal')

In [ ]:
pxx,pxy = np.meshgrid(np.arange(70,170),np.arange(30,100))

In [ ]:
path = mpath.Path(contour)

In [ ]:
pxx = pxx.reshape(-1,1)
pxy = pxy.reshape(-1,1)

In [ ]:
inPoly = path.contains_points(np.concatenate((pxx,pxy),axis=1))

In [ ]:
plt.scatter(pxx[~inPoly],pxy[~inPoly])

plt.scatter(pxx[inPoly],pxy[inPoly])

plt.axis('equal')

In [ ]:
pxx,pxy = np

In [ ]:
for c in b.keys():
    
#     plt.scatter(b[c][:,0],b[c][:,1])
    
    path = mpath.Path(b[c])
    
    inPoly = path.contains_points(np.concatenate((pxx,pxy),axis=1))
    
    plt.scatter(pxx[inPoly],pxy[inPoly],alpha = 0.2)
    
plt.axis('square')

In [ ]:
import pydicom as dcm

In [ ]:
dicom = dcm.dcmread('./pericardial/example/1.3.12.2.1107.5.2.18.41754.2015081410592855489077245.dcm',stop_before_pixels=False)

In [ ]:
dicom.file_meta

In [ ]:
plt.imshow(dicom.pixel_array,cmap = 'gray')

plt.scatter(a['freeDrawRoiContour'][:,0],a['freeDrawRoiContour'][:,1],c='r',s=1)

So, it appears that contours and images are in a matched coordinate space. The issue is going to be whether "real" scaling is necessary for this project...

In [ ]:
#now try the other one, with 3 contours


plt.figure(figsize = (30,30))

dicom = dcm.dcmread('./pericardial/example/1.3.12.2.1107.5.2.18.41754.201508141059285567277192.dcm',stop_before_pixels=False)

plt.imshow(dicom.pixel_array,cmap='gray')

for c in b.keys():
    
    plt.scatter(b[c][:,0],b[c][:,1],s=1)


In [ ]:
[px,py] = np.meshgrid(*[range(x) for x in dicom.pixel_array.shape])

px = px.reshape(-1,1)
py = py.reshape(-1,1)

pxy = np.concatenate([px,py],axis=1)

HOW TO CHECK FOR OFF-BY-ONE ERRORS?

if there were an off-by-one error due to Python zero-index and cvi42 one-indexing, then we would expect the cvi42 coordinates to be 1 higher than the correct python coordinates, so we would subtract one to get them back in frame.

let's have a look at this comparison...

In [ ]:
plt.figure(figsize = (30,30))

dicom = dcm.dcmread('./pericardial/example/1.3.12.2.1107.5.2.18.41754.201508141059285567277192.dcm',stop_before_pixels=False)

plt.imshow(dicom.pixel_array,cmap='gray')

for c in b.keys():
    
    plt.scatter(b[c][:,0],b[c][:,1],s=1,c='r',label = 'correct')
    
    
    plt.scatter(b[c][:,0]-1,b[c][:,1]-1,s=1,c='g',label = 'off-by-one')
    
    
plt.legend()


In [ ]:
plt.figure(figsize = (30,30))

dicom = dcm.dcmread('./pericardial/example/1.3.12.2.1107.5.2.18.41754.2015081410592855489077245.dcm',stop_before_pixels=False)

plt.imshow(dicom.pixel_array,cmap='gray')

for c in a.keys():
    
    plt.scatter(a[c][:,0],a[c][:,1],s=1,c='r',label = 'correct')
    
    
    plt.scatter(a[c][:,0]-1,a[c][:,1]-1,s=1,c='g',label = 'off-by-one')
    
    
    
plt.legend()


Well, that's not very fucking helpful is it.

In [ ]:
dicom[('0028','0030')]

In [ ]:
#lets try that again but using the origin and pixel spacing...

origin_key = ('0020', '0032')

orientation_key = ('0020', '0037')

pixelSize_key = ('0028','0030')

origin = dicom[origin_key]